In [1]:
# Import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

2.19.0-dev20241026


In [2]:
# Get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2024-10-28 21:46:16--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 104.26.2.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv.5’

train-data.tsv.5    100%[===================>] 349.84K  --.-KB/s    in 0.02s   

2024-10-28 21:46:16 (14.4 MB/s) - ‘train-data.tsv.5’ saved [358233/358233]

--2024-10-28 21:46:16--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 104.26.2.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv.5’

valid-data.tsv.5    100%[========

In [3]:
# Load and prepare the data
train_df = pd.read_csv(train_file_path, sep="\t", header=None, names=["type", "msg"])
test_df = pd.read_csv(test_file_path, sep="\t", header=None, names=["type", "msg"])
train_df["type"] = pd.factorize(train_df["type"])[0]
test_df["type"] = pd.factorize(test_df["type"])[0]
train_labels = train_df["type"].values
test_labels = test_df["type"].values

In [4]:
# Initialize and fit the tokenizer
tokenizer = Tokenizer(oov_token="<OOV>")
tokenizer.fit_on_texts(train_df["msg"])

In [5]:
# Convert text to sequences and add padding
train_sequences = tokenizer.texts_to_sequences(train_df["msg"])
test_sequences = tokenizer.texts_to_sequences(test_df["msg"])
maxlen = max(train_df["msg"].apply(lambda x: len(x.split())))
train_sequences = pad_sequences(train_sequences, maxlen=maxlen, padding='post')
test_sequences = pad_sequences(test_sequences, maxlen=maxlen, padding='post')

In [6]:
# Create the model
model = Sequential()
model.add(Embedding(len(tokenizer.word_index) + 1, 50))
model.add(Flatten())
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))

In [7]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.003), loss='binary_crossentropy', metrics=['accuracy'])

In [8]:
# Train the model
model.fit(
    train_sequences,
    train_labels,
    validation_data=(test_sequences, test_labels),
    epochs=35,
    verbose="auto"
)

AttributeError: module 'ml_dtypes' has no attribute 'float8_e3m4'
Epoch 1/35
131/131 ━━━━━━━━━━━━━━━━━━━━ 7s 29ms/step - accuracy: 0.8847 - loss: 0.2719 - val_accuracy: 0.9813 - val_loss: 0.0601
Epoch 2/35
131/131 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - accuracy: 0.9875 - loss: 0.0370 - val_accuracy: 0.9885 - val_loss: 0.0362
Epoch 3/35
131/131 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.9968 - loss: 0.0164 - val_accuracy: 0.9899 - val_loss: 0.0326
Epoch 4/35
131/131 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - accuracy: 0.9980 - loss: 0.0132 - val_accuracy: 0.9885 - val_loss: 0.0405
Epoch 5/35
131/131 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - accuracy: 0.9997 - loss: 0.0053 - val_accuracy: 0.9907 - val_loss: 0.0351
Epoch 6/35
131/131 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.9998 - loss: 0.0072 - val_accuracy: 0.9899 - val_loss: 0.0413
Epoch 7/35
131/131 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.9995 - loss: 0.0067 - val_accuracy: 0.9921 - val_loss: 0.0346
Epoch 8/35
131/131 ━━━━━━━━━━━━

In [9]:
# Evaluate the model
evaluation_metrics = model.evaluate(test_sequences, test_labels, verbose="auto")
print('Evaluation Results -> Loss:', evaluation_metrics[0], ', Accuracy:', evaluation_metrics[1])

44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9899 - loss: 0.1008
Evaluation Results -> Loss: 0.09555444121360779 , Accuracy: 0.9892241358757019


In [10]:
# Function to return prediction
def predict_message(pred_text):

    if not isinstance(pred_text, str) or len(pred_text.strip()) == 0:
        raise ValueError("Input must be a non-empty string.")

    seq = tokenizer.texts_to_sequences([pred_text])
    seq = pad_sequences(seq, maxlen=maxlen, padding='post', truncating='pre')

    prediction_score = model.predict(seq)[0, 0]
    label = "ham" if prediction_score < 0.5 else "spam"

    return [prediction_score, label]

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
[3.6263295e-12, 'ham']


In [11]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
You passed the challenge. Great job!
